Antes de empezar no olvides tener instalado python, pandas, numpy, pyomo y glpk

Iniciamos importando las librerias que necesitaremos y verificando si nuestro solver está disponible, en nuestro caso utilizamos GLPK

In [13]:
from pyomo.opt import SolverFactory
import pyomo.environ as pyo
import pandas as pd
import numpy as np

# Verifica si GLPK está disponible
SolverFactory('glpk').available()


True

Cargamos nuestra información

In [14]:
df = pd.read_excel('data/Equipo2.xlsx', engine='openpyxl')
df = df.rename(columns={'Unnamed: 0': 'Barcos'})
df

,Barcos,Puerto1,Puerto2,Puerto3,Puerto4
0,Barco1,5,4,6,7
1,Barco2,6,6,7,5
2,Barco3,7,5,7,6
3,Barco4,5,4,6,6


Ahora creamos el modelo que planteamos con anterioridad

In [15]:
# Conjunto de Barcos
B = df['Barcos'].unique()

# Conjunto de Puertos
P = df.iloc[:,1:].columns.unique() # Con esto obtenemos los encabezados de todas las columnas menos la de los barcos

# Costo total por enviar el barco b al puerto p
c = df.groupby(by=['Barcos']).sum()

# Creamos una variable que tendra nuestro modelo, usaremos uno del tipo Concreto
model = pyo.ConcreteModel()

# Creamos nuestra variable de decision
model.x = pyo.Var(B, P, within=pyo.Binary, doc='Asignacion de barcos a puertos')

# Establecemos nuestra funcion objetivo
model.obj = pyo.Objective(expr=sum(c.loc[b,p]*model.x[b,p] for b in B for p in P))

# Restricciones
# Cada barco va a un solo puerto
model.con1 = pyo.ConstraintList()
for b in B:
  model.con1.add(expr=sum(model.x[b, p] for p in P) == 1)

# A cada puerto llega solo un barco
model.con2 = pyo.ConstraintList()
for p in P:
  model.con2.add(expr=sum(model.x[b, p] for b in B) == 1)

Vamos a ver cómo quedó nuestro modelo

In [16]:
model.pprint()

1 Var Declarations
    x : Asignacion de barcos a puertos
        Size=16, Index={Barco3, Barco2, Barco1, Barco4}*{Puerto2, Puerto4, Puerto1, Puerto3}
        Key                   : Lower : Value : Upper : Fixed : Stale : Domain
        ('Barco1', 'Puerto1') :     0 :  None :     1 : False :  True : Binary
        ('Barco1', 'Puerto2') :     0 :  None :     1 : False :  True : Binary
        ('Barco1', 'Puerto3') :     0 :  None :     1 : False :  True : Binary
        ('Barco1', 'Puerto4') :     0 :  None :     1 : False :  True : Binary
        ('Barco2', 'Puerto1') :     0 :  None :     1 : False :  True : Binary
        ('Barco2', 'Puerto2') :     0 :  None :     1 : False :  True : Binary
        ('Barco2', 'Puerto3') :     0 :  None :     1 : False :  True : Binary
        ('Barco2', 'Puerto4') :     0 :  None :     1 : False :  True : Binary
        ('Barco3', 'Puerto1') :     0 :  None :     1 : False :  True : Binary
        ('Barco3', 'Puerto2') :     0 :  None :     1 : Fal

Llamamos a nuestro solver y resolvemos el problema

In [17]:
solver = pyo.SolverFactory('glpk')
resultados = solver.solve(model)

¡Es hora de ver los resultados!

In [18]:
for b in B:
    for p in P:
        if pyo.value(model.x[b, p]) != 0:
            print('Se debe enviar al barco {} al puerto {}, su costo es de {}'.format(b, p, pyo.value(c.loc[b, p])))

Se debe enviar al barco Barco1 al puerto Puerto1, su costo es de 5
Se debe enviar al barco Barco2 al puerto Puerto4, su costo es de 5
Se debe enviar al barco Barco3 al puerto Puerto2, su costo es de 5
Se debe enviar al barco Barco4 al puerto Puerto3, su costo es de 6


El costo minimo total para los cuatro envios es de...

In [19]:
print('{} unidades monetarias'.format(pyo.value(model.obj)))

21.0 unidades monetarias
